In [ ]:
#%% import libraries from local directory
import os
import sys
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
from os.path import relpath
from pybufrkit.decoder import Decoder
from geopandas.tools import sjoin
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.ops import nearest_points 
import requests
from datetime import datetime, timedelta
from pathlib import Path
from dateutil import rrule
import re
import xml.etree.ElementTree as ET
decoder = Decoder()
#wor_dir="/home/fbf/"
wor_dir="C:\\Users\\ATeklesadik\\OneDrive - Rode Kruis\\Documents\\documents\\Typhoon-Impact-based-forecasting-model"


### Hindcast analysis

In [ ]:


import os
 
import re

import pandas as pd
import xml.etree.ElementTree as ET

 
import requests
from datetime import datetime, timedelta
from pathlib import Path

from dateutil import rrule





#wor_dir='c:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/'
wor_dir="C:\\Users\\ATeklesadik\\OneDrive - Rode Kruis\\Documents\\documents\\Typhoon-Impact-based-forecasting-model"
typhoon_metadata_filename = os.path.join(wor_dir, "IBF-Typhoon-model\\data\\metadata_typhoons.csv")
typhoon_metadata = pd.read_csv(typhoon_metadata_filename, delimiter=",")

 
typhoon_metadata['landfall_dd_hh']=typhoon_metadata.apply(lambda x:x.landfalldate +' '+ x.landfall_time,axis=1	)
typhoon_metadata['landfall_']=typhoon_metadata.apply(lambda x:datetime.strptime(x.landfall_dd_hh, '%d/%m/%Y %H:%M:00'),axis=1	)
 
 
typhoon_metadata=typhoon_metadata.query('typhoon in @list_of_events')
typhoon_metadata['landfall_dd_hh']=typhoon_metadata.apply(lambda x:x.landfalldate +' '+ x.landfall_time,axis=1	)
typhoon_metadata['landfall_']=typhoon_metadata.apply(lambda x:datetime.strptime(x.landfall_dd_hh, '%d/%m/%Y %H:%M:00'),axis=1	)
typhoon_metadata=typhoon_metadata.query('typhoon in @list_of_events')

list_of_events=[
    'krosa2013','fengshen2008','haiyan2013','rammasun2014', 'hagupit2014',  'haima2016','phanfone2019', 
    'nock-ten2016' 
    ]



 
typhoon_metadata["international"] = typhoon_metadata['typhoon'].apply(lambda x:x[:-4])
typhoon_metadata["year"] = typhoon_metadata['landfall_'].apply(lambda x:x.year)

 
 
df_typhoons = typhoon_metadata

### download and process input data 

In [ ]:


def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour) - timedelta(hours=t.hour-6))

#from constants import save_dir


tem_dir="C:\\Users\\ATeklesadik\\OneDrive - Rode Kruis\\Documents\\documents\\Typhoon-Impact-based-forecasting-model\\IBF-Typhoon-model\\data\\wind_data\\"
save_dir=os.path.join(tem_dir,'temp')
save_dir = Path(save_dir)

email = input('email: ')
pswd = input('password: ')

values = {'email' : email, 'passwd' : pswd, 'action' : 'login'}
login_url = 'https://rda.ucar.edu/cgi-bin/login'

ret = requests.post(login_url, data=values)


values = {'email' : email, 'passwd' : pswd, 'action' : 'login'}
login_url = 'https://rda.ucar.edu/cgi-bin/login'

ret = requests.post(login_url, data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print(ret.text)
    exit(1)

dspath = 'https://rda.ucar.edu/data/ds330.3/'

verbose = True


for date,row in typhoon_metadata.iterrows():
    start_date =row['landfall_'] - timedelta(hours=72)
    start_date =hour_rounder(start_date)
    end_date = row['landfall_'] 
    
    date_list = rrule.rrule(rrule.HOURLY, 
                            dtstart=start_date, 
                            until=end_date,
                            interval=6)
    
    for date in date_list:
        ymd = date.strftime("%Y%m%d")
        ymdhms = date.strftime("%Y%m%d%H%M%S")
        server = "test" if date < datetime(2008, 8, 1) else "prod"
        file = f'ecmf/{date.year}/{ymd}/z_tigge_c_ecmf_{ymdhms}_ifs_glob_{server}_all_glo.xml'
        filename = dspath + file
        outfile = save_dir / "xml" / os.path.basename(filename)
        # Don't download if exists already
        if outfile.exists():
            if verbose:
                print(f'{file} already exists')
            continue
        req = requests.get(filename, cookies = ret.cookies, allow_redirects=True)
        if req.status_code != 200:
            if verbose:
                print(f'{file} invalid URL')
            continue
        if verbose:
            print(f'{file} downloading')
        open(outfile, 'wb').write(req.content)
  

def xml2csv(filename):
    print(f"{filename}")
    try:
        tree = ET.parse(filename)
    except ET.ParseError:
        print("Error with file, skipping")
        return
    root = tree.getroot()

    prod_center=root.find('header/productionCenter').text
    baseTime=root.find('header/baseTime').text

    ## Create one dictonary for each time point, and append it to a list
    for members in root.findall('data'):
        mtype=members.get('type')
        if mtype not in ['forecast', 'ensembleForecast']:
            continue
        for members2 in members.findall('disturbance'):
            cyclone_name = [name.text.lower().strip() for name in members2.findall('cycloneName')]
            if not cyclone_name:
                continue
            cyclone_name = cyclone_name[0].lower()
            if cyclone_name not in list(df_typhoons["international"]):
                continue
            print(f"Found typhoon {cyclone_name}")
            for members3 in members2.findall('fix'):
                tem_dic = {}
                tem_dic['mtype']=[mtype]
                tem_dic['product']=[re.sub('\s+',' ',prod_center).strip().lower()]
                tem_dic['cyc_number'] = [name.text for name in members2.findall('cycloneNumber')]
                tem_dic['ensemble']=[members.get('member')]
                tem_dic['speed'] = [name.text for name in members3.findall('cycloneData/maximumWind/speed')]
                tem_dic['pressure'] = [name.text for name in members3.findall('cycloneData/minimumPressure/pressure')]
                time = [name.text for name in members3.findall('validTime')]
                tem_dic['time'] = ['/'.join(time[0].split('T')[0].split('-'))+', '+time[0].split('T')[1][:-1]]
                tem_dic['lat'] = [name.text for name in members3.findall('latitude')]
                tem_dic['lon']= [name.text for name in members3.findall('longitude')]
                tem_dic['lead_time']=[members3.get('hour')]
                tem_dic['forecast_time'] = ['/'.join(baseTime.split('T')[0].split('-'))+', '+baseTime.split('T')[1][:-1]]
                tem_dic1 = dict( [(k,''.join(str(e).lower().strip() for e in v)) for k,v in tem_dic.items()])
                # Save to CSV
                outfile = save_dir / f"csv/{cyclone_name}_all.csv"
                pd.DataFrame(tem_dic1, index=[0]).to_csv(outfile, mode='a', header=not os.path.exists(outfile), index=False)


 

# Get list of filenames
filename_list = sorted(list(Path(save_dir / "xml").glob('*.xml')))

for filename in filename_list:
    xml2csv(filename)


### calculate wind field data for each manucipality 

In [ ]:
typhoon_forecast_dict={}

list_of_events=['nock-ten2016']
filename_list1=[fname for fname in filename_list if os.path.basename(fname) in ['nock-ten_2016.csv']]

import time
for i,row, in typhoon_metadata.iterrows():
    landfall_=row['landfall_']
    landfall_time=datetime.strptime(landfall_.strftime('%Y/%m/%d, %H:%M:%S'), '%Y/%m/%d,  %H:%M:%S') 
    typhoon_name=row['typhoon']
    if typhoon_name in list_of_events:#'hagupit2014':  
        tr_data=[tr for tr in DATA_YPHOON1 if (tr.name+tr.sid[:4])==typhoon_name]
        forecast_time_list=sorted(list(set([tr.forecast_time  for tr in tr_data])), reverse=False)
        for forecast_time in forecast_time_list:
            lead_time1=(landfall_time-forecast_time).total_seconds() / 3600 
            if lead_time1 > 23:                
                tr_data2=[tr for tr in tr_data if tr.forecast_time==forecast_time]
                tracks = TCTracks()
                # Select typhoons that are in the typhoon event sheet
                tracks.data =tr_data2 # [ tr for tr in DATA_YPHOON if (tr.name + tr.sid[:4]) in typhoon_events]
                #tracks.data = [adjust_tracks(tr) for tr in Typhoons.data]
                tracks.equal_timestep(0.5)
                
                TYphoon = TropCyclone()
                TYphoon.set_from_tracks(tracks, cent, store_windfields=True,metric="geosphere")
                windfield=TYphoon.windfields
                #calculate wind field for each ensamble members 
                df = pd.DataFrame(data=cent.coord)
                df["centroid_id"] = "id" + (df.index).astype(str)
                centroid_idx = df["centroid_id"].values
                ncents = cent.size
                df = df.rename(columns={0: "lat", 1: "lon"})
                threshold = 0.1

                df_ = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
                df_.crs = {"init": "epsg:4326"}
                df_ = df_.to_crs("EPSG:4326")
                df_admin = sjoin(df_, admin, how="left")
                # To remove points that are in water
                df_admin = df_admin.dropna()

                list_intensity = []
                distan_track = []

                list_intensity=[]
                distan_track=[]
                
                for i in range(len(tracks.data)):        
                    nsteps = windfield[i].shape[0]
                    tr=tracks.data[i]
                    centroid_id = np.tile(centroid_idx, nsteps)
                    intensity_3d = windfield[i].toarray().reshape(nsteps, ncents, 2)
                    intensity = np.linalg.norm(intensity_3d, axis=-1).ravel()
                    timesteps = np.repeat(tracks.data[i].time.values, ncents)
                    timesteps = timesteps.reshape((nsteps, ncents)).ravel()
                    inten_tr = pd.DataFrame({
                            'centroid_id': centroid_id,
                            'value': intensity,
                            'timestamp': timesteps,})
                    
                    inten_tr = inten_tr[inten_tr.value > threshold]
                    
                    inten_tr['storm_id'] = tr.sid
                    #inten_tr['ens_id'] =tr.sid+'_'+str(tr.ensemble_number)
                    inten_tr['name'] = tr.name
                    
                    inten_tr = (pd.merge(inten_tr, df_admin, how='outer', on='centroid_id')
                                .dropna()
                                .groupby(['adm3_pcode'], as_index=False)
                                .agg({"value": ['count', 'max']}))
                    
                    
                    inten_tr.columns = [x for x in ['adm3_pcode', 'value_count', 'v_max']]
                    inten_tr['storm_id'] = tr.sid
                    inten_tr['name'] = tr.name
                    inten_tr['forecast_time']=tr.forecast_time
                    inten_tr['lead_time']=lead_time1
                    list_intensity.append(inten_tr)
 
                
        
                    distan_track1=[]
                    
                    for index, row in df.iterrows():
                        dist=np.min(np.sqrt(np.square(tr.lat.values-row['lat'])+np.square(tr.lon.values-row['lon'])))
                        distan_track1.append(dist*111)
                    dist_tr = pd.DataFrame({'centroid_id': centroid_idx,'value': distan_track1})

                    dist_tr = (pd.merge(dist_tr, df_admin, how='outer', on='centroid_id')
                                .dropna()
                                .groupby(['adm3_pcode'], as_index=False)
                                .agg({'value': 'min'}))
                    

                    
                    dist_tr.columns = [x for x in ['adm3_pcode', 'dis_track_min']]  # join_left_df_.columns.ravel()]
                    dist_tr['storm_id'] = tr.sid
                    #dist_tr['name'] = tr.name
                    distan_track.append(dist_tr)
    
                df_intensity_ = pd.concat(list_intensity)
                distan_track1 = pd.concat(distan_track)

                typhhon_df =  pd.merge(df_intensity_, distan_track1,  how='left', on=['adm3_pcode','storm_id']) 

                # Check if there are duplicates for municipality and storm_id


                #distan_track1 = pd.concat(distan_track)

                #typhhon_df = df_intensity_ #pd.merge(df_intensity_, distan_track1,  how='left', on=['adm3_pcode','storm_id']) 

                # Check if there are duplicates for municipality and storm_id

                duplicate = typhhon_df[
                    typhhon_df.duplicated(subset=["adm3_pcode", "storm_id"], keep=False)
                ]



                    
                file_name = (f"IBF-typhoon-model\\data\\wind_data\\hindcast\\hindcast_{typhoon_name}_{lead_time1}.csv")

                path = os.path.join(wor_dir, file_name)

                typhhon_df.to_csv(path)    

### Modeling  

In [ ]:
# standard libraries
import datetime as dt
from pathlib import Path

import pandas as pd
import numpy as np
import os
# additional libraries
import cartopy.crs as ccrs
import cftime
import geopandas as gpd
import matplotlib.cm as cm_mp
from matplotlib.collections import LineCollection
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np

import scipy.io.matlab as matlab
from shapely.geometry import Point, LineString, MultiLineString
from sklearn.neighbors import DistanceMetric
import statsmodels.api as sm
import xarray as xr

import random
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier

from sklearn.feature_selection import RFECV
import pandas as pd
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    StratifiedKFold,
    train_test_split,
    KFold)
 
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import (
    mean_absolute_error, 
    mean_squared_error,
    recall_score,
    f1_score,
    precision_score,
    confusion_matrix,
    make_scorer)

import xgboost as xgb
import glob

from geopandas.tools import sjoin
 
from shapely.ops import nearest_points
import datetime as datetime

wor_dir="C:\\Users\\ATeklesadik\\OneDrive - Rode Kruis\\Documents\\documents\\Typhoon-Impact-based-forecasting-model"
f_path = os.path.join(wor_dir,"IBF-typhoon-model/data/all_predisaster_indicators.csv") 
df_predisasters = pd.read_csv(f_path)


def division(x, y):
    try:
        value =100* (x / y)
        
    except:
        value = np.nan
    
    return 100 if value>100 else value


# Setting the new damage threshold

df_predisasters["vulnerable_groups"] = df_predisasters.apply(lambda x: division(x["vulnerable_groups"], x["Total Pop"]), axis=1).values
#df_total["pantawid_pamilya_beneficiary"] = df_total.apply(lambda x: division(x["Total # of Active HHs"], x["Housing Units"]), axis=1).values
df_predisasters["pantawid_pamilya_beneficiary"] = df_predisasters.apply(lambda x: division(x["pantawid_total_pop"], x["Total Pop"]), axis=1).values
#%%
#############################



 
combined_input_data=pd.read_csv(os.path.join(wor_dir,"IBF-typhoon-model/data/model_input/df_modelinput_sep.csv")) 
tphoon_events=combined_input_data[['typhoon','DAM_perc_dmg']].groupby('typhoon').size().to_dict()
### for hyper parameter optimization we looped over typhoon events with at least 100 data entries  

 
def set_zeros(x):
    x_max = 25
    y_max = 50    
    v_max = x[0]
    rainfall_max = x[1]
    damage = x[2]
    if pd.notnull(damage):
        value = damage
    elif v_max > x_max or rainfall_max > y_max:
        value =damage
    elif (v_max < np.sqrt((1- (rainfall_max**2/y_max ** 2))*x_max ** 2)):
        value = 0
    #elif ((v_max < x_max)  and  (rainfall_max_6h < y_max) ):
    #elif (v_max < x_max ):
    #value = 0
    else:
        value = np.nan
    return value

combined_input_data["DAM_perc_dmg"] = combined_input_data[["HAZ_v_max", "HAZ_rainfall_Total", "DAM_perc_dmg"]].apply(set_zeros, axis="columns")


selected_features_xgb_regr= [#'HAZ_rainfall_max_24h',
                                       'HAZ_v_max',
                                       'HAZ_dis_track_min',
                                       'TOP_mean_slope',
                                       'TOP_mean_elevation_m',
                                       'TOP_ruggedness_stdev',
                                       'TOP_mean_ruggedness',
                                       'TOP_slope_stdev',
                                       'VUL_poverty_perc',
                                       'GEN_with_coast',
                                       'VUL_Housing_Units',
                                       'VUL_StrongRoof_StrongWall',
                                       'VUL_StrongRoof_LightWall',
                                       'VUL_StrongRoof_SalvageWall',
                                       'VUL_LightRoof_StrongWall',
                                       'VUL_LightRoof_LightWall',
                                       'VUL_SalvagedRoof_StrongWall',
                                       'VUL_SalvagedRoof_LightWall',
                                       'VUL_SalvagedRoof_SalvageWall',
                                       'VUL_vulnerable_groups',
                                       'VUL_pantawid_pamilya_beneficiary']



# split data into train and test sets

SEED2 = 314159265
SEED = 31 
 
test_size = 0.1

# Full dataset for feature selection

combined_input_data_ = combined_input_data[combined_input_data['DAM_perc_dmg'].notnull()]


X = combined_input_data_[selected_features_xgb_regr]
y = combined_input_data_["DAM_perc_dmg"]
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=SEED)





#from sklearn.metrics import mean_absolute_error

 
reg = xgb.XGBRegressor(base_score=0.5,
             booster='gbtree', 
             subsample=0.8,
             eta=0.05,
             max_depth=8, 
             colsample_bylevel=1,
             colsample_bynode=1, 
             colsample_bytree=1,
             early_stopping_rounds=10, 
             eval_metric=mean_absolute_error,
             gamma=1, 
             objective='reg:squarederror',
             gpu_id=-1, 
             grow_policy='depthwise', 
             learning_rate=0.025,
             min_child_weight=1,
             n_estimators=200,        
             random_state=42,
             tree_method="hist",
             )

eval_set=[(X_train, y_train), ( X_test, y_test)]

reg.fit(X, y, eval_set=eval_set)

#%%%

 
 


#%%%%%%%%

#Adding the wind data only for events with impact data


In [ ]:
dref_probabilities = {
            "100k": [100000, 0.5],
            "80k": [80000, 0.6],
            "70k": [70000, 0.7],
            "50k": [50000, 0.8],
            "30k": [30000, 0.9],
        }


dref_probabilities_10 = {
            "33": [33,'Moderate'],
            "50": [50,'High'],
            "70":[70,'Very High'],
        }


cerf_probabilities = {
    "80k": [80000, 0.5],
    "50k": [50000, 0.6],
    "30k": [30000, 0.7],
    "10k": [10000, 0.8],
    "5k": [5000, 0.95],
}
        
        
START_probabilities = {
    'PH166700000':{
        "8k": [8000, 0.8],
        "10k": [10000, 0.8],
        "15k": [15000, 0.7],
        "20k": [20000, 0.5],
        "25k": [25000, 0.5],    
         
        },
    'PH021500000':{
        "16k": [16000, 0.8],
        "18k": [18000, 0.8],
        "20k": [20000, 0.7],
        "25k": [25000, 0.5],
        "28.5k": [28500, 0.5], 
   
        },
    'PH082600000':{
        "18k": [18000, 0.8],
        "20k": [20000, 0.8],
        "25k": [25000, 0.7],
        "30k": [30000, 0.6],
        "40k": [40000, 0.5],
        },
}

# Only select regions 5 and 8
cerf_regions = ["PH05", "PH08"]

start_regions = {'PH166700000':'surigaodelnorte', 'PH021500000':'cagayan', 'PH082600000':'EasternSamar'}

cerf_trigger_list={}
start_trigger_dic={}
DREF_trigger_list={}
DREF_trigger_list_10={}


file_name = ("IBF-typhoon-model\\data\\wind_data\\output\\")#historical_events_windgrid_output.csv")
#historical_events_windgrid_output_hrs
wind_path = os.path.join(wor_dir, file_name)
#wind_path = os.path.join(wor_dir,"data/wind_data/output_storm2")
#os.chdir(wind_path)

dfs=[]
def Number_affected(x):
    return np.exp(6.80943612231606) * x ** 0.46982114400549513
    
def division(x, y):
    try:
        value = int((x * y)/100)
        
    except:
        value = np.nan
    
    return x if value>x else value 

filename_list_wind = sorted(list(Path(os.path.join(wor_dir ,"IBF-typhoon-model/data/wind_data/hindcast")).glob('*.csv')))
inx_=0
index2=0
inx_3=0
 
for file_wind in filename_list_wind:
    #windfiles = glob.glob(f'*_{index}.csv')
    index=os.path.basename(file_wind)
    index_=index.split('.')[0]
    event_name=index.split('_')[1]
    lead_time2=index.split('_')[-1].split('.')[0]
    #f_path = os.path.join(wind_path,f'{index}.csv')
 
    
    df_temp = pd.read_csv(file_wind)
    
    #if len(df_temp.index)>1:
    df_temp.rename(columns={'adm3_pcode':'Mun_Code','name':'typhoon'},inplace=True)
        
    df_wind=df_temp[['storm_id','Mun_Code','v_max','typhoon','forecast_time','dis_track_min']]#.groupby("Mun_Code").agg({'v_max':'max'})
     
 
    df_wind_final = df_wind#pd.concat(dfs,axis=0, join='outer')    
    df_total=pd.merge(df_wind_final, df_predisasters,  how='left', left_on='Mun_Code', right_on = 'Mun_Code')
    df_total=df_total[df_total['v_max'].notnull()]
    df_total.rename(columns ={           'v_max':'HAZ_v_max',
                                        'dis_track_min':'HAZ_dis_track_min',
                                        'landslide_per':'GEN_landslide_per',
                                        'stormsurge_per':'GEN_stormsurge_per',
                                        'Bu_p_inSSA':'GEN_Bu_p_inSSA',
                                        'Bu_p_LS':'GEN_Bu_p_LS',
                                         'Red_per_LSbldg':'GEN_Red_per_LSbldg',
                                        'Or_per_LSblg':'GEN_Or_per_LSblg',
                                         'Yel_per_LSSAb':'GEN_Yel_per_LSSAb',
                                        'RED_per_SSAbldg':'GEN_RED_per_SSAbldg',
                                         'OR_per_SSAbldg':'GEN_OR_per_SSAbldg',
                                        'Yellow_per_LSbl':'GEN_Yellow_per_LSbl',
                                         'mean_slope':'TOP_mean_slope',
                                        'mean_elevation_m':'TOP_mean_elevation_m',
                                         'ruggedness_stdev':'TOP_ruggedness_stdev',
                                        'mean_ruggedness':'TOP_mean_ruggedness',
                                         'slope_stdev':'TOP_slope_stdev',
                                         'poverty_perc':'VUL_poverty_perc',
                                        'with_coast':'GEN_with_coast',
                                         'coast_length':'GEN_coast_length',
                                         'Housing Units':'VUL_Housing_Units',
                                        'Strong Roof/Strong Wall':"VUL_StrongRoof_StrongWall",
                                        'Strong Roof/Light Wall':'VUL_StrongRoof_LightWall',
                                        'Strong Roof/Salvage Wall':'VUL_StrongRoof_SalvageWall',
                                        'Light Roof/Strong Wall':'VUL_LightRoof_StrongWall',
                                        'Light Roof/Light Wall':'VUL_LightRoof_LightWall',
                                        'Light Roof/Salvage Wall':'VUL_LightRoof_SalvageWall',
                                        'Salvaged Roof/Strong Wall':'VUL_SalvagedRoof_StrongWall',
                                        'Salvaged Roof/Light Wall':'VUL_SalvagedRoof_LightWall',
                                        'Salvaged Roof/Salvage Wall':'VUL_SalvagedRoof_SalvageWall',
                                        'vulnerable_groups':'VUL_vulnerable_groups',
                                        'pantawid_pamilya_beneficiary':'VUL_pantawid_pamilya_beneficiary'},inplace=True)
    #df_total = df_total.filter(selected_cols)
    X_all = df_total[selected_features_xgb_regr]
    y_pred = reg.predict(X_all)
    
    
    df_total['DMG_predicted']=y_pred 
    y_pred[y_pred<10]=0
    y_pred[y_pred>0]=1
    df_total['Trigger']=y_pred 
    
    IMPACT_DF1 = pd.merge(df_total, df_predisasters[['Housing Units','Mun_Code']],  how='left', left_on='Mun_Code', right_on = 'Mun_Code') 
    IMPACT_DF1["number_dmg_Build_prediction"] = IMPACT_DF1.apply(lambda x: division(x["Housing Units"], x["DMG_predicted"]), axis=1).values
  
    
    IMPACT_DF1["number_affected_pop__prediction"] = IMPACT_DF1.apply(lambda x: Number_affected(x["number_dmg_Build_prediction"]), axis=1).values
    
    impact_scenarios=['number_dmg_Build_prediction','number_affected_pop__prediction']
    
    IMPACT_DF1[impact_scenarios] = IMPACT_DF1[impact_scenarios].astype('int') 
    IMPACT_DF1['Prov_Code']=IMPACT_DF1.apply(lambda x:str(x.Mun_Code[:6])+'00000',axis=1)
    IMPACT_DF1['typhoon_year']=IMPACT_DF1.apply(lambda x:x.typhoon+str(x.storm_id[:4]),axis=1)
    
    
    
     
    impact_path = os.path.join(wor_dir,"IBF-typhoon-model/data/results/hindcast",f"{index}impact.csv")
 
    

    
    
    IMPACT_DF12=IMPACT_DF1[['Prov_Code','Mun_Code','storm_id','typhoon_year', 'forecast_time','DMG_predicted','Trigger','number_dmg_Build_prediction','number_affected_pop__prediction']]
    IMPACT_DF12.to_csv(impact_path) 
    
    dfs.append(IMPACT_DF12)
    
    
    probability_impact=IMPACT_DF12.groupby(['storm_id','typhoon_year','forecast_time']).agg(NUmber_of_affected_municipality=('Mun_Code','count'),
                                        average_ML_Model=('DMG_predicted', 'mean'),
                                        Trigger_ML_Model=('Trigger', sum),
                                        Total_affected_ML_Model=('number_affected_pop__prediction', sum),
                                        Total_buildings_ML_Model=('number_dmg_Build_prediction', sum)).sort_values(by='Total_buildings_ML_Model',ascending=False).reset_index()
    
  
    
    
    ### DREF trigger based on 10% damage per manucipality  
    trigger_status1={}
    probability_impact['Trigger']=probability_impact.apply(lambda x:1 if x.Trigger_ML_Model>2 else 0,axis=1)
    agg_impact_10 = probability_impact["Trigger"].values
    trigger_stat_dref10 = 100*(sum(agg_impact_10) /len(agg_impact_10))
    
    for key, values in dref_probabilities_10.items():
        inx_3=inx_3+1
        dref_trigger_status10 = {}
        thershold=values[1]
        if  (trigger_stat_dref10 > values[0]):
            trigger_stat_1=True
 
        else:
            trigger_stat_1=False
 
        dref_trigger_status10['lead_time'] = int(lead_time2)-10
        dref_trigger_status10['triggered_prob'] = thershold 
        dref_trigger_status10['EVENT'] = event_name 
        dref_trigger_status10['trigger_stat'] = trigger_stat_1         
        trigger_status1[inx_3] = dref_trigger_status10
        index3=event_name+'_'+lead_time2+str(inx_3)
        DREF_trigger_list_10[index3] = dref_trigger_status10
        
    
    ### DREF trigger based on probability of Houses 
    agg_impact = probability_impact["Total_buildings_ML_Model"].values
    for key, values in dref_probabilities.items():
        dref_trigger_status = {}
        index2=index2+1

        trigger_stat = 100*(sum(1.13*i > values[0] for i in agg_impact) /len(agg_impact))
        
        if  (values[1] < (sum(1.13*i > values[0] for i in agg_impact) /len(agg_impact))):
            trigger_stat_=True
            thershold=key 
        else:
            trigger_stat_=False
            thershold=key
        
        dref_trigger_status['thre_name'] = key
        dref_trigger_status['thr_value'] = int(trigger_stat)      
        dref_trigger_status['event'] = event_name 
        dref_trigger_status['lead_time'] = int(lead_time2)-10
        dref_trigger_status['trigger_stat'] = trigger_stat_
        index=event_name+'_'+lead_time2+str(index2)
        
        DREF_trigger_list[index] = dref_trigger_status
    
        
    #DREF_trigger_list[event_name] = dref_trigger_status
    
    
    
    ############ check trigger for START project
    
    start_trigger_status = {}
    
    provinces_names={'PH166700000':'SurigaoDeLnorte','PH021500000':'Cagayan','PH082600000':'EasternSamar'}
    
    for provinces in ['PH166700000','PH021500000','PH082600000']:
        triggers=START_probabilities[provinces]
        prov_name=provinces_names[provinces]
        df_trig=IMPACT_DF12.query('Prov_Code==@provinces')
        if not df_trig.empty:  
            probability_impact=df_trig.groupby(['storm_id','typhoon_year','forecast_time']).agg(
                                                Total_affected_ML_Model=('number_affected_pop__prediction', sum),
                                                Total_buildings_ML_Model=('number_dmg_Build_prediction', sum)).sort_values(by='Total_buildings_ML_Model',ascending=False).reset_index()
            
                
            ######## calculate probability for impact
            
            
            agg_impact = probability_impact["Total_affected_ML_Model"].values
     
            thershold='NAN'
            
            

            for key, values in triggers.items():
                inx_=inx_+1
                start_trigger_status1 = {}
                trigger_stat = 100*(sum(i > values[0] for i in agg_impact) /len(agg_impact)       )
                #start_trigger_status1[key] = int(trigger_stat)
                start_trigger_status1['thr_name']=key
                start_trigger_status1['thr_value']=int(trigger_stat)

                if  (values[1] < (sum(i > values[0] for i in agg_impact) /len(agg_impact))):
                    trigger_stat_=True
                    thershold=key 
                else:
                    trigger_stat_=False
                    thershold=key
            #start_trigger_list.append({'event':event_name, 'province':prov_name,                                      'lead_time':lead_time2,                                      'status':trigger_stat_,                                      'threshold':thershold})
            
                #start_trigger_status['event'] = event_name
 

                start_trigger_status1['PROVINCE'] = prov_name
                start_trigger_status1['EVENT'] = event_name
                start_trigger_status1['lead_time'] = lead_time2
                start_trigger_status1['trigger_stat'] = trigger_stat_         
                index=event_name+'_'+lead_time2+str(index2)
                start_trigger_dic[index] = start_trigger_status1
       
            #start_trigger[prov_name] =start_trigger_status
            
            #start_trigger_list.append(start_trigger_status)
    #start_trigger_list[event_name] = start_trigger_status
     

Trigger_status=True
impact_path_ = os.path.join(wor_dir,"IBF-typhoon-model/data/results/dref_trigger_table_10%.csv")
pd.DataFrame.from_dict(DREF_trigger_list_10,orient='index').query('trigger_stat==@Trigger_status').to_csv(impact_path_) 

impact_path_ = os.path.join(wor_dir,"IBF-typhoon-model/data/results/dref_trigger_table_prob.csv")
pd.DataFrame.from_dict(DREF_trigger_list,orient='index').query('trigger_stat==@Trigger_status').to_csv(impact_path_) 

impact_path_ = os.path.join(wor_dir,"IBF-typhoon-model/data/results/start_trigger_table_prob.csv")
pd.DataFrame.from_dict(start_trigger_dic,orient='index').query('trigger_stat==@Trigger_status').to_csv(impact_path_) 

 
  